# Interactive Dashboard for Cuisine Types and Ingredients
Data sourced from Kaggle and CIA.gov are processed here and plotted in an interactive map. The outputs hosted online. Author: Marc Petta 
## Data Processing

In [1]:
import json 
import pandas as pd

# read in data 
df1 = pd.read_json (r'data/train.json')
df2 = pd.read_csv('data/diet.csv')
# convert string in these columns to lowercase
df1["cuisine"].str.lower()
df1["ingredients"].str.lower()
# check for nulls
df1.isnull().values.any()

False

In [2]:
# explode ingredient list
df1 = df1.explode('ingredients')
# review
df1.head()

,id,cuisine,ingredients
0,10259,greek,romaine lettuce
0,10259,greek,black olives
0,10259,greek,grape tomatoes
0,10259,greek,garlic
0,10259,greek,pepper


In [3]:
# review
df2.tail()

,cuisine,latitude,longitude,adult_obesity_rate,health_expenditures,education_expenditures,last_update
15,japanese,35.652832,139.839478,4.3,10.2,3.8,1/1/20
16,irish,53.350140,-6.266155,25.3,7.8,6.4,1/1/20
17,korean,37.532600,127.024612,4.7,7.4,5.0,1/1/20
18,moroccan,33.589886,-7.603869,26.1,5.9,5.4,1/1/20
19,russian,55.751244,37.618423,23.1,7.1,4.1,1/1/20


In [4]:
# get unique values for cuisine
df1['cuisine'].unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

In [5]:
# how many ingredients are in the recipes
len(df1['ingredients'].unique())

6714

In [6]:
# get list of all ingredients
ingredients = df1['ingredients'].unique().tolist()
len(ingredients)

6714

That is alot of ingredients. The potential exists for some of the strings in that colummn to be synonymous with one another. One way we can check os by using the fuzzywuzzy package which checks for string similarity using Levenshtein distance. Levenshtein distance is a metric to measure the difference in two sequences of words. It is the minimum number of single-character edits required to change one word into the other.

In [ ]:
from fuzzywuzzy import process, fuzz

# get list of all ingredients
items = df1['ingredients'].unique().tolist()

processedItems = []
matchers = []

for item in items:
    if item:
        processedItem = fuzz._process_and_sort(item, True, True)
        processedItems.append({"processed": processedItem, "item": item})


processedItems.sort(key= lambda x: len(x["processed"]))

for idx, item in enumerate(processedItems):
    length = len(item["processed"])
    matcher = fuzz.SequenceMatcher(None, item["processed"])
    for idx2 in range(idx + 1, len(processedItems)):
        item2 = processedItems[idx2]
        if 2 * length / (length + len(item2["processed"])) < 0.85: # upper bound
            break

        matcher.set_seq2(item2["processed"])

        if matcher.quick_ratio() >= 0.99 and matcher.ratio() >= 0.99: # should also try without quick_ratio() check
            print(item["item"], item2["item"])


This next procedure is a bit verbose but the data does require some cleaning at this stage.

In [7]:
# replace strings in ingredients columns to reduce synonomous items
df1["ingredients"] = df1["ingredients"].str.replace("garlic cloves", "garlic")
df1["ingredients"] = df1["ingredients"].str.replace("cummin", "cumin")
df1["ingredients"] = df1["ingredients"].str.replace("ground cumin", "cumin")
df1["ingredients"] = df1["ingredients"].str.replace("chillies", "chiles")
df1["ingredients"] = df1["ingredients"].str.replace("culantro", "cilantro")
df1["ingredients"] = df1["ingredients"].str.replace("cardamon", "cardamom")
df1["ingredients"] = df1["ingredients"].str.replace("corn flour", "cornflour")
df1["ingredients"] = df1["ingredients"].str.replace("cuminseed", "cumin")
df1["ingredients"] = df1["ingredients"].str.replace("cumin seed", "cumin")
df1["ingredients"] = df1["ingredients"].str.replace("gingerroot", "ginger")
df1["ingredients"] = df1["ingredients"].str.replace("ginger root", "ginger")
df1["ingredients"] = df1["ingredients"].str.replace("chili paste", "chile paste")
df1["ingredients"] = df1["ingredients"].str.replace("blackpepper", "black pepper")
df1["ingredients"] = df1["ingredients"].str.replace("stem ginger", "ginger")
df1["ingredients"] = df1["ingredients"].str.replace("basil leaves", "basil")
df1["ingredients"] = df1["ingredients"].str.replace("chili pepper", "chiles")
df1["ingredients"] = df1["ingredients"].str.replace("ground pepper", "black pepper")
df1["ingredients"] = df1["ingredients"].str.replace("ground peppercorn", "black pepper")
df1["ingredients"] = df1["ingredients"].str.replace("serrano chile", "chiles")
df1["ingredients"] = df1["ingredients"].str.replace("serrano chilies", "chiles")
df1["ingredients"] = df1["ingredients"].str.replace("diced tomatoes", "tomato")
df1["ingredients"] = df1["ingredients"].str.replace("dried tomatoes", "tomato")
df1["ingredients"] = df1["ingredients"].str.replace("grape tomatoes", "tomato")
df1["ingredients"] = df1["ingredients"].str.replace("sliced tomatoes", "tomato")
df1["ingredients"] = df1["ingredients"].str.replace("roasted garlic", "garlic")
df1["ingredients"] = df1["ingredients"].str.replace("jamaican jerk", "jerk seasoning")
df1["ingredients"] = df1["ingredients"].str.replace("jerk seasoning mix", "jerk seasoning")
df1["ingredients"] = df1["ingredients"].str.replace("jamaican jerk rub", "jerk seasoning")
df1["ingredients"] = df1["ingredients"].str.replace("jerk rub seasoning", "jerk seasoning")
df1["ingredients"] = df1["ingredients"].str.replace("ground tumeric", "turmeric")
df1["ingredients"] = df1["ingredients"].str.replace("ground turmeric", "turmeric")
df1["ingredients"] = df1["ingredients"].str.replace("diced bell pepper", "bell pepper")
df1["ingredients"] = df1["ingredients"].str.replace("mixed bell peppers", "bell pepper")
df1["ingredients"] = df1["ingredients"].str.replace("green bell pepper", "bell pepper")
df1["ingredients"] = df1["ingredients"].str.replace("green bell pepper, slice", "bell pepper")
df1["ingredients"] = df1["ingredients"].str.replace("granulated sugar", "sugar")
df1["ingredients"] = df1["ingredients"].str.replace("fine granulated sugar", "sugar")
df1["ingredients"] = df1["ingredients"].str.replace("cane sugar", "sugar")
df1["ingredients"] = df1["ingredients"].str.replace("grated parmesan cheese", "parmesan cheese")
df1["ingredients"] = df1["ingredients"].str.replace("parmesan", "parmesan cheese")
df1["ingredients"] = df1["ingredients"].str.replace("ground black pepper", "black pepper")
df1["ingredients"] = df1["ingredients"].str.replace("extra-virgin olive oil", "olive oil")
df1["ingredients"] = df1["ingredients"].str.replace("extra virgin olive oil", "olive oil")
df1["ingredients"] = df1["ingredients"].str.replace("feta cheese crumbles", "feta cheese")
df1["ingredients"] = df1["ingredients"].str.replace("fresh lemon juice", "lemon juice")
df1["ingredients"] = df1["ingredients"].str.replace("parmesan cheese cheese", "parmesan cheese")
df1["ingredients"] = df1["ingredients"].str.replace("fresh basil", "basil")
df1["ingredients"] = df1["ingredients"].str.replace("kosher salt", "salt")

Salt and pepper are rather universally used ingredients and simply arent that interesting here. We will remove all the rows that have these ingredients for all cuisines.

In [8]:
# get the count of ingredients for comparison
len(df1.ingredients)

428275

In [9]:
# Get names of indexes for which column ingredients has value salt
indexNames = df1[ df1['ingredients'] == 'salt' ].index
# Delete these row indexes
df1.drop(indexNames , inplace=True)
len(df1.ingredients)

# Get names of indexes for which column ingredients has value black pepper
indexNames2 = df1[ df1['ingredients'] == 'black pepper' ].index
# Delete these row indexes 
df1.drop(indexNames2 , inplace=True)
len(df1.ingredients)

# Get names of indexes for which column ingredients has value black pepper
indexNames2 = df1[ df1['ingredients'] == 'water' ].index
# Delete these row indexes 
df1.drop(indexNames2 , inplace=True)
len(df1.ingredients)

125159

In [10]:
# get only the top five ingredients for each cuisine group
df4 = df1['ingredients'].groupby(df1['cuisine']).value_counts()
df5 = df4.groupby(level=0).nlargest(5).reset_index(level=0, drop=True)

df5

cuisine     ingredients             
brazilian   cachaca                      65
            sugar                        64
            lime                         58
            sweetened condensed milk     46
            garlic                       27
                                       ... 
vietnamese  garlic                      126
            fish sauce                  121
            sugar                        95
            carrots                      66
            soy sauce                    54
Name: ingredients, Length: 100, dtype: int64

The previous group by call results in a panda series with a hierarchical index. This needs to be undone and  converted back to a data frame. This can be done with the following. 

In [11]:
# convert series to data frame and reset index
df5 = df5.to_frame(name="nas").reset_index() 
df5

,cuisine,ingredients,nas
0,brazilian,cachaca,65
1,brazilian,sugar,64
2,brazilian,lime,58
3,brazilian,sweetened condensed milk,46
4,brazilian,garlic,27
...,...,...,...
95,vietnamese,garlic,126
96,vietnamese,fish sauce,121
97,vietnamese,sugar,95
98,vietnamese,carrots,66


In [12]:
# group by again to get a list of each ingredients per each cuisine
df6 = (df5.groupby('cuisine')['ingredients']
       .apply(lambda x: list(set(x)))
       .reset_index())

Now that we have processed the data we can join the datasets to have our final dataframe to plot

In [13]:
# merge each dataframe on cuisine
df = df6.merge(df2, left_on='cuisine', right_on='cuisine')
df

,cuisine,ingredients,latitude,longitude,adult_obesity_rate,health_expenditures,education_expenditures,last_update
0,brazilian,"[sugar, sweetened condensed milk, cachaca, gar...",-23.533773,-46.625290,22.1,8.3,5.8,1/1/20
1,british,"[sugar, all-purpose flour, butter, unsalted bu...",51.509865,-0.118092,27.8,9.1,6.2,1/1/20
2,cajun_creole,"[olive oil, cajun seasoning, onions, butter, g...",29.951065,-90.071533,30.0,17.1,5.4,1/1/20
3,chinese,"[sugar, corn starch, sesame oil, soy sauce, ga...",39.913818,116.363625,6.2,5.5,0.0,1/1/20
4,filipino,"[sugar, carrots, onions, soy sauce, garlic]",16.566233,121.262634,6.4,4.7,2.7,1/1/20
5,french,"[olive oil, sugar, butter, garlic, unsalted bu...",48.864716,2.349014,21.6,11.5,5.9,1/1/20
6,greek,"[olive oil, lemon juice, garlic, dried oregano...",37.983810,23.727539,24.9,8.1,4.1,1/1/20
7,indian,"[cumin, onions, garam masala, vegetable oil, g...",25.794033,78.116531,3.9,4.7,3.2,1/1/20
8,irish,"[sugar, all-purpose flour, potatoes, butter, e...",53.350140,-6.266155,25.3,7.8,6.4,1/1/20
9,italian,"[parmesan cheese, olive oil, basil, onions, ga...",42.504154,12.646361,19.9,9.2,4.5,1/1/20


In [14]:
# write dataframe to csv for plotting
df.to_csv('dietFull.csv', index=False)

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('data/dietFull.csv')

total_cuisine = 20
total_recipe = 39774
total_ingredients = 428275

df_top10 = df.nlargest(10, "adult_obesity_rate")
top10_countries_1 = df_top10["cuisine"].tolist()
top10_obesity = df_top10["adult_obesity_rate"].tolist()

df_top10 = df.nlargest(10, "health_expenditures")
top10_countries_2 = df_top10["cuisine"].tolist()
top10_health = df_top10["health_expenditures"].tolist()

df_top10 = df.nlargest(10, "education_expenditures")
top10_countries_3 = df_top10["cuisine"].tolist()
top10_education = df_top10["education_expenditures"].tolist()

fig = make_subplots(
    rows = 4, cols = 6,

    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, 
             {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],[    None, None, None,               
             {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              
             {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               
             {"type": "bar", "colspan":3}, None, None],
          ]
)

message = df["cuisine"] + "<br>"
message += "Most Common Ingredients: " + df["ingredients"].astype(str) + "<br>"
message += "Adult Obesity Rates: " + df["adult_obesity_rate"].astype(str) + "<br>"
message += "Country Health Expenditures: " + df["health_expenditures"].astype(str) + "<br>"
message += "Country Education Expenditures: " + df["education_expenditures"].astype(str)
df["text"] = message

fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = df["longitude"],
        lat = df["latitude"],
        hovertext = df["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(97, 153, 209)'
            ),
            cmin = 0,
            color = df['adult_obesity_rate'],
            cmax = df['adult_obesity_rate'].max(),
            colorbar_title="Adult Obesity Rates",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_cuisine,
        title="Cuisine Types",
    ),
    row=1, col=4
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recipe,
        title="Recipes Analyzed",
    ),
    row=1, col=5
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_ingredients,
        title="Total Count of Ingredients",
    ),
    row=1, col=6
)

fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_obesity, 
        name= "Adult Obesity Rates",
        marker=dict(color="lightcoral"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_health, 
        name= "Health Expenditures",
        marker=dict(color="navajowhite"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_education, 
        name= "Education Expenditures",
        marker=dict(color="steelblue"), 
        showlegend=True),
    row=4, col=4
)


fig.update_layout(
    template="seaborn",
    title = "Recipe Analysis by Cuisine Type with Health Related Data",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="ghostwhite", 
            showland= True,
            showocean = False,
            lakecolor="aliceblue"
    ),

    annotations=[
        dict(
            text="Source: https://www.cia.gov/library/publications/resources/the-world-factbook/" + "<br>" + "https://www.kaggle.com/kaggle/recipe-ingredients-dataset/data",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=-0.05)
    ]
)

fig.write_html('dietDash.html', auto_open=True)